In [1]:
%%html

<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

### American revolution 

<img src="images/13_colonies.jpg" width="400px" align="right"/>
The American Revolution (1775-83) is also known as the American Revolutionary War and the U.S. War of Independence.

Independance means not beeing under the control of others and beeing able to make one's own decisions. On July 4, 1776,  the Thirteen Colonies declared that they were no longer part of Britain. The British disagreed. A war between Britain and the Thirteen Coloniws began as a result of this disagreement. This war was known as the American Revolution or the War of Independence. (From "Our Land and People", Patricia Shields-Ramsay, Douglas Ramsay)



Lets download and  explore open dataset with information about American revolution battles.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy import nan as Nan
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
import time
init_notebook_mode(connected=True)

url = 'https://data.opendatasoft.com/explore/dataset/list-of-american-revolutionary-war-battles@public-us/download/?format=csv&timezone=America/Denver&use_labels_for_header=true'
revolution_data  = pd.read_csv(url,sep=';',error_bad_lines=False)
revolution_data.head()

,Battle,article,Date,State,Outcome,Start Date,Victory,Battle location
0,Battles of Lexington and Concord,https://en.wikipedia.org/wiki/Battles_of_Lexin...,"April 19, 1775",Massachusetts,Patriot victory: British forces raiding Concor...,1775-04-19,Patriot victory,"42.4496388889, -71.2308333333"
1,First Siege of Fort Henry,https://en.wikipedia.org/wiki/Siege_of_Fort_He...,NaN,Virginia,Patriot victory,NaN,Patriot victory,"40.0638888889, -80.725"
2,Battle of Freeman's Farm,https://en.wikipedia.org/wiki/Battles_of_Saratoga,"September 19, 1777",New York,British tactical victory: First of the two Bat...,1777-09-19,NaN,"42.9988888889, -73.6375"
3,Siege of Augusta,https://en.wikipedia.org/wiki/Siege_of_Augusta,NaN,Georgia,Patriot victory,NaN,Patriot victory,"33.4666666667, -81.9666666667"
4,Battle of Videau's Bridge,https://en.wikipedia.org/wiki/Battle_of_Videau...,"January 2, 1782",South Carolina,British victory,1782-01-02,British victory,NaN


Look at the last column - Battle location is presented with two numbers - latitude and longitude. 
Let's take a step back and explore what is latitude and longitude.

**Latitudes and longitudes** are very basic terms in geography.
They help us to explain the location or position of a place on the globe. 
Latitudes and longitudes form a grid system on the globe, by knowing this we an easily find out any place on the earth.

#### Latitudes

Latitudes are horizontal imaginary lines on the globe.
Latitudes are also known as "parallels" as they are at an equal distance from each other and never meet.

Latitude lines are a numerical way to measure how far north or south of the equator a place is located. The equator is the starting point for measuring latitude--that's why it's marked as 0 degrees latitude. The number of latitude degrees will be larger the further away from the equator the place is located, all the way up to 90 degrees latitude at the poles. Latitude locations are given as **degrees North**(+) or **degrees South**(-).

Each degree of latitude is approximately 111 kilometers apart (69 miles),although there is little variation as the earth is not perfectly round as it is shaped slightly like an egg.


In [3]:
contours = []

equator_lon = [-180,-170,-160,-150,-140,-130,-120,-110,-100,-90,-80,-70,-60,-50,-40,-30,-20,-10,0,
               10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180]
euator_lat = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

#equator
contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= '0° equator',
        name = '0° equator',
        lon = equator_lon,
        lat = euator_lat,
        mode = 'lines',
        line = dict(
            width = 3,
            color = 'green'
        )
    ) )
#north pole
contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= '90° north pole',
        name = '90° north pole',
        lon = [0],
        lat = [90],
        mode = 'markers',
        line = dict(
            width = 2,
            color = 'rgb(50,136,189)'
        )
    ) )
#north hemispere
for i in range (1,9):
    contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= str(10*i)+'° north',
        name = str(10*i)+'° north',
        lon = equator_lon,
        lat = [x+10*i for x in euator_lat],
        mode = 'lines',
        line = dict(
            width = 2,
            color = 'rgb(50,136,189)'
        )
    ) )
    
#south hemispere
for i in range (1,9):
    contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= str(-10*i)+'° south',
        name = str(-10*i)+'° south',
        lon = equator_lon,
        lat = [x-10*i for x in euator_lat],
        mode = 'lines',
        line = dict(
            width = 2,
            color = 'rgb(244,109,67)'
        )
    ) )
    
contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= '-90° south pole',
        name = '-90° south pole',
        lon = [0],
        lat = [-90],
        mode = 'markers',
        line = dict(
            width = 2,
            color = 'rgb(244,109,67)'
        )
    ) )

layout = dict(
        title = 'Parallels over globe<br>(Click and drag to rotate)',        
        geo = dict(
            showland = True,
            showlakes = True,
            showcountries = True,
            showocean = True,
            countrywidth = 0.5,
            lakecolor = 'rgb(255, 255, 255)',
            oceancolor = 'rgb(255, 255, 255)',
            landcolor = 'rgb(212, 212, 212)',
            projection = dict( 
                type = 'orthographic',
                rotation = dict(
                    lon = -100,
                    lat = 40,
                    roll = 0
                )            
            ),

        )
    )
    
fig = dict( data=contours, layout=layout )
iplot( fig, validate=False )

#### Questions

Look at the globe try rotating it and hover your mouse over different parallels. 
We displayed every 10th parallel
If you click on the legend for any parallel on the left - if will disappear/reappear on the map.

Can you find Equator? Which continents does it go through?
Prallels located south to the Equator are colored orange. and  this part of the Earth is called Southern Hemisphere.
Prallels located north to the Equator are colored blue. and  the part of the Earth  they cover is called Northern Hemisphere.

Find Canada. Is it located in Southern or Northern Hemispehere?
Try to define inbetween of which parallels is Canada located?

#### Longitudes
The vertical lines on the globe are known as longitude.
They all converge at poles and are widest at equator i.e.at the middle of the earth.
They are also known meridians.
Longitude lines are a numerical way to show/measure how far a location is east or west of a universal vertical line called the Prime Meridian. This Prime Meridian line runs vertically, north and south, right over the British Royal Observatory in Greenwich England, from the North Pole to the South Pole. As the vertical starting point for longitude, the Prime Meridian is numbered 0 degrees longitude.

To measure longitude east or west of the Prime Meridian, there are 180 vertical longitude lines east of the Prime Meridian and 180 vertical longitude lines west of the Prime Meridian, so longitude locations are given as **degrees east**(+) or **degrees west**(-). The 180 degree line is a single vertical line called the International Date Line, and it is directly opposite of the Prime Meridian.

In [4]:
contours = []

mer_lat1 = [90,100,110,120,130,140,150,160,170,180,-180,-170,-160,-150,-140,-130,-120,-110,-100,-90]
mer_lon1 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

mer_lat2 = [-90,-80,-70,-60,-50,-40,-30,-20,-10,0,10,20,30,40,50,60,70,80,90]
mer_lon2 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

#zero meridian
contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= '0° Prime meridian',
        name = '0° Prime meridian',
        lon = mer_lon2,
        lat = mer_lat2,
        mode = 'lines',
        line = dict(
            width = 3,
            color = 'green'
        )
    ) )

contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= '180° International Date Line',
        name = '180° International Date Line',
        lon = mer_lon1,
        lat = mer_lat1,
        mode = 'lines',
        line = dict(
            width = 3,
            color = 'green'
        )
    ) )

#west hemispere
for i in range (1,18):
    contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= str(10*i-180)+'° west',
        name = str(10*i-180)+'° west',
        lat = mer_lat1,
        lon = [x+10*i for x in mer_lon1],
        mode = 'lines',
        line = dict(
            width = 2,
            color = 'rgb(244,109,67)'
        )
    ) )
    
#east hemispere
for i in range (1,18):
    contours.append( dict(
        type = 'scattergeo',
        hoverinfo='text',
        text= str(10*i)+'° east',
        name = str(10*i)+'° east',
        lat = mer_lat2,
        lon = [x+10*i for x in mer_lon2],
        mode = 'lines',
        line = dict(
            width = 2,
            color = 'rgb(50,136,189)'
        )
    ) )
    


layout = dict(
        title = 'Meridians over globe<br>(Click and drag to rotate)',
       # showlegend = False,         
        geo = dict(
            showland = True,
            showlakes = True,
            showcountries = True,
            showocean = True,
            countrywidth = 0.5,
            lakecolor = 'rgb(255, 255, 255)',
            oceancolor = 'rgb(255, 255, 255)',
            landcolor = 'rgb(212, 212, 212)',
            projection = dict( 
                type = 'orthographic',
                rotation = dict(
                    lon = -100,
                    lat = 40,
                    roll = 0
                )            
            ),
    
        )
    )
    
fig = dict( data=contours, layout=layout )
iplot( fig, validate=False )

#### Questions

Look at the globe try rotating it and hover your mouse over different meridians. 
We displayed every 10th meridian.
If you click on the legend for any meridian on the left - if will disappear/reappear on the map.

Do you see the Prime Meridian? Can you see that its going though Great Britain/London?  Interestingly, for many years, each country had its own prime meridian. France used a meridian passing through Paris, while England used a meridian passing through Greenwich, England.Only in 1884, a group of scientists, navigators, and businessmen made Greenwich, England, the official worldwide Prime Meridian.

Meridians located east to the Prime Meridian are colored blue and  this part of the Earth is called Eastern Hemisphere.
Meridians located west to the Prime Meridian are colored orange and  this part of the Earth is called Western Hemisphere.

Look at Canada. Is it located in Eastern or Western Hemispehere?
Try to define - inbetween of which meridians is Canada located?

#### Looking at Canada's map
Great observations! 
Canada is located between -50 and -140 degrees West (longitude) and between 42 and 83 degrees North (latitude).
Lets display some of the Canadian cities on the map.

In [5]:
import plotly.plotly as py
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
df.head()
name=["Edmonton", "Victoria","Winnipeg","Fredericton","St. John's","Halifax","Toronto",
      #"Charlottetown","Quebec City", 
      "Regina","Whitehorse","Iqaluit","Yellowknife", "Ottawa"]
long=[-113.49,-123.37,-97.14,-66.64, -52.71, -63.57, -79.38,
      #-63.13,-71.21,
      -104.62,-135.06,-68.52,-114.38,-75.70]
latt=[53.54,48.43,49.90,45.97, 47.56, 44.65, 43.65,
      #46.238240,46.813878,
      50.44,60.72,63.75,62.45,45.42]
df['text'] = df['airport'] + '' + df['city'] + ', ' + df['state'] + '' + 'Arrivals: ' + df['cnt'].astype(str)

data = [ dict(
        type = 'scattergeo',
        locationmode = 'north america',
        lon = long,
        lat = latt,
        text = name,
        mode = 'markers+text',
        textposition= 'top',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            color = 'red',
        ))]

layout = dict(
        title = "Some Canada's provincial capitals <br>(Hover for coordinates)",
        colorbar = True,
        geo = dict(
            scope = 'north america',
            showland = True,
            landcolor = "rgb(212, 212, 212)",
            countrycolor = "rgb(255, 255, 255)",
            showlakes = True,
            lakecolor = "rgb(255, 255, 255)",
            showsubunits = False,
            showcountries = True,
            resolution = 50,
            projection = dict(
                type = 'kavrayskiy7',
            ),
            lonaxis = dict(
                showgrid = True,
                gridcolor ='rgb(244,109,67)',
                gridwidth = 2,
                range= [ -140, -50 ],
                dtick = 10
            ),
            lataxis = dict (
                showgrid = True,
                gridcolor ='rgb(50,136,189)',
                gridwidth = 2,
                range= [ 40.0, 80.0 ],
                dtick = 10
            )
        )
    )

fig = dict( data=data, layout=layout )
iplot( fig, validate=False )

#### Questions 
Look at the map, hover over cities. 
Latitude and logitude for each place are displayed as a pair - (lon, lat). 

For example Ottawa (-75.7, 45.42)
It means that Ottawa is located 75.7 degrees west (longitude)  and 45.42 degrees north (latitude).

Note that for west/south coordinates you can use minus(-) oe explicitly say west or south.

Look at other cities on the map - which of them is located more north? What are it's coordinates?

Look at the Winnipeg and Regina's coordinates. 
Their lattlitude are very close -  they lay almost on the same parallel!

Look at Edmonton and Yellowknife. What can you say abouyt theisr longitudes?
What is the difference between latitudes for Edmonton and Yellowknife?
Can you try calculating the distance between these cities given that 1 degree of latitude is covering approximately 111 km?

The distance is approximately (62.45-53.54)*111 = 989.01km 

#### Displaying battles on the map

After we learned about longitudae and latitude  - lets go back to our dataset with American revolution battles.

Let's split "Battle location" column into two: "lon" and "lat". Now we have coordintaes and can display battles on the map.
 We are interested as well to look at battle by year  - so let's create  "Year" column based on first 4 characters of the "Start Date" column.
In adition cleaning some empty values from the dataset(based on  "Battle location" and "Start Date" columns).

In [6]:
revolution_data = revolution_data.dropna(subset=['Battle location'])
revolution_data = revolution_data.dropna(subset=['Start Date'])
revolution_data['lat'] = revolution_data['Battle location'].str.split(', ', expand = True)[0]
revolution_data['lon'] = revolution_data['Battle location'].str.split(', ', expand = True)[1]
revolution_data['Year'] = revolution_data['Start Date'].str[:4]
revolution_data.head()

,Battle,article,Date,State,Outcome,Start Date,Victory,Battle location,lat,lon,Year
0,Battles of Lexington and Concord,https://en.wikipedia.org/wiki/Battles_of_Lexin...,"April 19, 1775",Massachusetts,Patriot victory: British forces raiding Concor...,1775-04-19,Patriot victory,"42.4496388889, -71.2308333333",42.4496388889,-71.2308333333,1775
2,Battle of Freeman's Farm,https://en.wikipedia.org/wiki/Battles_of_Saratoga,"September 19, 1777",New York,British tactical victory: First of the two Bat...,1777-09-19,NaN,"42.9988888889, -73.6375",42.9988888889,-73.6375,1777
5,Mount Hope Bay raids,https://en.wikipedia.org/wiki/Mount_Hope_Bay_r...,"May 25-30, 1778",Rhode Island,British victory,1778-05-25,British victory,"41.694, -71.216",41.694,-71.216,1778
6,Capture of Savannah,https://en.wikipedia.org/wiki/Capture_of_Savannah,"December 29, 1778",Georgia,British victory,1778-12-29,British victory,"32.0508333333, -81.1038888889",32.0508333333,-81.1038888889,1778
7,Grand Assault on Gibraltar,https://en.wikipedia.org/wiki/Great_Siege_of_G...,"September 13, 1782",Gibraltar,British victory,1782-09-13,British victory,"36.1397, -5.3551",36.1397,-5.3551,1782


Now we can try displaying battles on the map using their latitude and longitude. From the dataset we will display battles only for North America and remove outliner ( year 2026).
We will add an animated slider to separate battles by year.

In [15]:
revolution_data['lat']=revolution_data['lat'].astype(float)
revolution_data['lon']=revolution_data['lon'].astype(float)
revolution_data=revolution_data[revolution_data['lat']>12]
revolution_data=revolution_data[revolution_data['lat']<60]
revolution_data=revolution_data[revolution_data['lon']>-150]
revolution_data=revolution_data[revolution_data['lon']<-40]
revolution_data=revolution_data[revolution_data['Year'] != '2026']
revolution_data.loc[revolution_data.shape[0]]=[' ',' ',' ',' ',' ',' ','Patriot victory',' ','0','0','1783']

years_from_col = set(revolution_data['Year'])
years_ints = sorted(list(years_from_col))
years = [str(year) for year in years_ints]
revolution_data['Victory'] = revolution_data['Victory'].replace(np.nan, 'Unknown', regex=True)

victories = []

for victory in revolution_data['Victory']:
    if victory not in victories: 
        victories.append(victory)       

figure = {
    'data': [],
    'layout': {},
    'frames': []
}

figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1774',
    'plotlycommand': 'animate',
    'values': years,
    'visible': True
}

custom_colors = {
    'Patriot victory': 'rgb(171, 0, 250)',
    'Unknown': 'rgb(150, 99, 0)',
    'British victory': 'rgb(99, 110, 250)',
}

figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}
figure['layout']['title'] = ' American Revolution (1775-83) battles'
figure['layout']['geo']  =  dict(
        scope = 'north america',
        showland = True,
        landcolor = "rgb(212, 212, 212)",
        subunitcolor = "rgb(255, 255, 255)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = True,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation = dict(
                lon = -100
            )
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
)
 

year = 1774
for victory in victories:
    dataset_by_year = revolution_data[revolution_data['Year'] == year]
    dataset_by_year_and_vict = dataset_by_year[dataset_by_year['Victory'] == victory]

    data_dict = {
        'type': 'scattergeo',
        'lon': list(dataset_by_year_and_vict['lon']),
        'lat': list(dataset_by_year_and_vict['lat']),
        'mode': 'markers',
        'text': list(dataset_by_year_and_vict['Battle']),
        'marker': {
            'size': 8, 
            'color': custom_colors[victory]
        },
        'name': victory,
        'legendgroup' : 'group1'
    }
    figure['data'].append(data_dict)
    

for year in years:
    frame = {'data': [], 'name': str(year)}
    for victory in victories:
        dataset_by_year = revolution_data[revolution_data['Year'] == year]
        dataset_by_year_and_vict = dataset_by_year[dataset_by_year['Victory'] == victory]

        data_dict = {
            'type': 'scattergeo',
            'lon': list(dataset_by_year_and_vict['lon']),
            'lat': list(dataset_by_year_and_vict['lat']),
            'mode': 'markers',
            'text': list(dataset_by_year_and_vict['Battle']),
            'marker': {
                'size': 8, 
                'color': custom_colors[victory]
            },
            'name': victory,
            'legendgroup' : 'group'
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

    
figure['layout']['sliders'] = [sliders_dict]
iplot(figure, validate=False)

#### Questions.

Look at the map. Try hitting play button and see how revolution emerged. 

If you drag  the slider to the specific year you can see all the battles for this year.
If you hover the mouse ove the battle you can see its name, longitude and latitude and who won the battle.

Which year do you think had the most battles?

Try zooming in the map.
Which part of North America did the most battles happen?

#### Displaying some statistics.
We can check ourselves - lets display some statistics - the largest number of battles by year.

In [16]:
import colorlover as cl
results=revolution_data.groupby(['Year']).size().reset_index(name='Count').sort_values(by=['Count'])
results= results[results['Year']!=" "].reset_index(drop=True)
results["Colour"]=['rgb(255,255,255)'] + cl.scales['9']['seq']['Blues'] 
results=results.sort_values(by=['Year'])
trace0 = go.Table(
  type = 'table',
  header = dict(
    values = [ "<b>YEAR</b>","<b>NUMBER OF BATTLES</b>",],
    line = dict(color = 'white'),
    fill = dict(color = 'white'),
    align = ['center'],
    font = dict(color = 'black', size = 12)
  ),
  cells = dict(
    values = [results.Year, results.Count],
    line = dict(color = [results.Colour]),
    fill = dict(color = [results.Colour]),
    align = 'center',
    height = 30,
    font = dict(color = 'green', size = 20)
    ))

data = [trace0]

iplot(data)

We can see that year 1777 and 1780 - the darkest in the table  - have the largest number of battles 30 and 26.
Years 1774 and 1783 - the smallest number - 1 and 5.

Lets look at the number of battles by location:

In [17]:
results=revolution_data.groupby(['State']).size().reset_index(name='Count').sort_values(by=['Count'],ascending=False)
results= results[results['State']!=" "].reset_index(drop=True)
results['Percent'] = round(results['Count']*100/sum(results['Count']),2).astype(str) +"%"
trace0 = go.Bar(
    x=results['State'],
    y=results['Count'],
    text=results['Percent'],
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5,
        )
    ),
    opacity=0.6
)

data = [trace0]
layout = go.Layout(
    title='Number of battles by location',
   # font=dict(size=18),
    margin = dict(
    l= 30,
    r= 30,
    t= 50,
    b= 250
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

If you hover the mouse ove the plot - you can see that the most battles occured states New York -27(16,7%), South Carolina -26(15.57%) and New Jersey -23(13.77%).

### Summary

We have learned  about longitude, latitude and how to determine the absolute location of places in Canada on maps and globes. We have tried to use historical maps to make meaning of American revolution. We have observed how the revolution emerged year by year on a map and collected some interesting statistics about the battle location/frequency.

As the last step we can explore one of the naval battles of American revolution that took place in Canada.

#### Naval battle off Halifax
<img src="images/Halifax_battle.jpg" width="400px" align="right"/>
The Battle off Halifax took place on 28 May 1782 during the American Revolutionary War. It involved the American privateer Jack and the 14-gun Royal Naval brig HMS Observer off Halifax, Nova Scotia. Captain David Ropes commanded Jack, and Lieutenant John Crymes commanded Observer. The battle was "a long and severe engagement."

Observer was returning to Halifax, having rescued from American privateer Noah Stoddard ten crew members of HMS Blonde after Blonde wrecked on Seal Island off Cape Sable Island.[9] On 28 May 1782, as Observer arrived at the Sambro Island Light near the mouth of Halifax Harbour, Jack approached her. When Jack discovered her quarry was a British naval vessel, the Americans tried to escape.

Observer chased Jack for two hours before catching her. The ships were evenly matched. The British immediately killed the American captain David Ropes as a result of the cannon fire. Both ships had numerous holes shot through their sails, and the British sailors attempted to climb the rigging of their ship in an effort to board the American privateer. The Americans repulsed this initial boarding attempt, but the British were ultimately successful. Jack struck her colours on the afternoon of 29 May.